# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

/usr/bin/sh: 1: {sys.executable}: not found


import findspark
findspark.init('/home/elias/spark-3.0.0-preview2-bin-hadoop2.7/')
import pyspark
import numpy

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Hunday').getOrCreate()

In [12]:
from pyspark.ml.regression import LinearRegression
df = spark.read.csv("cruise_ship_info.csv",inferSchema=True, header=True)

In [19]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [21]:
# create an dummy variable for Cruise_line
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol='Cruise_line',outputCol='CL_Cat')
df_indexed = indexer.fit(df).transform(df)
df_indexed.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)
 |-- CL_Cat: double (nullable = false)



In [34]:
#count unique values
df_indexed.groupBy('Cruise_line').count().orderBy('count').show()


+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|           Orient|    1|
|          Crystal|    2|
|          Azamara|    2|
|           Disney|    2|
|           Cunard|    3|
|          Oceania|    3|
|         Seabourn|    3|
|         Windstar|    3|
|        Silversea|    4|
|Regent_Seven_Seas|    5|
|             Star|    6|
|              P&O|    6|
|              MSC|    8|
|        Celebrity|   10|
|            Costa|   11|
|        Norwegian|   13|
| Holland_American|   14|
|         Princess|   17|
|         Carnival|   22|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [35]:
df_indexed.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)
 |-- CL_Cat: double (nullable = false)



In [46]:
# Import VectorAssembler and Vectors
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(
    inputCols=["CL_Cat", "Age", 
               "Tonnage",'passengers','length','cabins','passenger_density',],
    outputCol="features")

Assem_DF = assembler.transform(df_indexed)

In [47]:
Final_DF= Assem_DF.select("features","crew")

In [48]:
#Split the data into validaiton and test set
train_df , test_df = Final_DF.randomSplit([0.7,0.3])

In [56]:
#Build the linear regression model
from pyspark.ml.regression import LinearRegression
LR = LinearRegression(labelCol='crew')
LR_model= LR.fit(train_df)

In [57]:
Test_res = LR_model.evaluate(test_df)

In [59]:
print("RMSE: {}".format(Test_res.rootMeanSquaredError))
print("MSE: {}".format(Test_res.meanSquaredError))
print("R2: {}".format(Test_res.r2))

RMSE: 1.5161004355078542
MSE: 2.2985605305471055
R2: 0.8436003834647436


In [66]:
from pyspark.sql.functions import corr
df.select(corr('crew',('passengers' ))).show()

+----------------------+
|corr(crew, passengers)|
+----------------------+
|    0.9152341306065384|
+----------------------+

